# 콜금리 crawling
crawling 범위: 논문구현에 필요한 2005년부터 2019년까지의 콜금리(callrate)   
requests와 BeautifulSoup 모듈을 사용해서 crawling 하는 코드입니다. 

issue: 콜금리는 매일매일 열리지는 않기 때문에, 한달 이전의 콜금리 자료가 존재하지 않을 수 있다!   
이를 어떻게 해결해야 할 것인가?? 

In [1]:
# import moduels
import requests 
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import time # 걸린 시간 측정 
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [42]:
# %%writefile C:\Users\User\Documents\python_basic\paper_implementation\module_name.py
# Naver 콜금리는 지속적으로 업데이트 되기 때문에, 수집하고자 하는 범위를 항상 확인하고 실행할 것 
class callrate_package(object):
    callrate_list = [] # callrate를 담을 빈 리스트 생성

    def crawling_callrate(self, start, end): # start와 end는 page number
        self.start = start
        self.end = end
        
        for i in range(start, end): # for 문을 통해 page 접근 
            print(i,'번 째 페이지를 가져오고 있습니다...')
            url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}'.format(i)

            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            content = soup.find('table', class_='tbl_exchange today').find_all('tr') # tag를 찾아 접근 

            for td in content:
                temp_dict = {}

                date = td.find('td', class_='date') # tag를 찾아 접근 
                if date is not None: # None 값들을 제외하고 수집
                    date = date.text.replace('\n','').replace('\t','') # 개행문자 \n, \t 삭제
                    temp_dict['date'] = date
                    # print(date)

                callrate = td.find('td', class_='num')
                if callrate is not None:
                    callrate = callrate.text
                    temp_dict['callrate'] = float(callrate)
                    # print(callrate)

                    callrate_package.callrate_list.append(temp_dict)

    def labeling_callrate(self):
        df = pd.DataFrame(callrate_package.callrate_list) # df 생성
        df['old_date'] = None # 한달전 date
        df['old_callrate'] = None # 한달전 콜금리
        df['updown'] = None # labeling
        
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index(df['date'])
        
#         df['date'] = pd.to_datetime(df['date']).apply(lambda x: x.date()).values
#         df['old_date'] = pd.to_datetime(df['old_date']).apply(lambda x: x.date()).values
        
#         # type 찍어보기 
#         print(type(df['old_date'][1]))
#         print(type(df['date'][1]))
#         print(df['date'])
#         print(list(df['date']))

        for i in range(len(df.index)):
    
            if df['date'][i] >= pd.Timestamp('2005-01-01T00'):
    #             print(i) # 정상적으로 찍힘 0~ 62

                df['old_date'][i] = df['date'][i] - pd.DateOffset(months = 1)
    #             df['date'] = pd.to_datetime(df['date']).apply(lambda x: x.date()).values
    #             df['old_date'] = pd.to_datetime(df['old_date']).apply(lambda x: x.date()).values
    #             print(type(df['old_date'][i]))
    #             print(type(df['date'][i]))
    #             print(type(df.index[i]))
    #             print(df['old_date'][0])
    #             print(df.index)
#                 count = 0 

                while df['old_date'][i] not in df['date']:
#                     count += 1
    #                 df['old_date'][i] = df['old_date'][i] - pd.Timedelta(days = 1)
                    df['old_date'][i] = df['old_date'][i] - pd.DateOffset(days=1)
#                     print("while문 이에요", df['old_date'][i])
#     #                 print(type(df['old_date'][i]))
#                     print(count)
    #                     if df['old_date'][i] <= pd.Timestamp('2005-01-01T00'):
    #                         continue

                # old_date의 index의 값을 가져온다.    
                if df['old_date'][i] in df.index:
                    df['old_callrate'][i] = df.loc[df['old_date'][i]]['callrate']
                    print("돌고있어요", df['old_date'][i])

    #                 if df['old_date'][i] in df.index:
    #                     df['old_callrate'][i] = df.loc[df['old_date'][i]]['callrate']

                try:
                    if (df['callrate'][i] - df['old_callrate'][i]) < 0:
                        df['updown'][i] = 'down'
                    elif (df['callrate'][i] - df['old_callrate'][i]) == 0:
                        df['updown'][i] = 'neutrality'
                    else:
                        df['updown'][i] = 'up'
                except:
                    pass
                
#                 continue

        print(df)
        return df.to_csv("callrate1.csv")
        
    

In [32]:
aa = callrate_package()

In [33]:
aa.crawling_callrate(1, 530)

1 번 째 페이지를 가져오고 있습니다...
2 번 째 페이지를 가져오고 있습니다...
3 번 째 페이지를 가져오고 있습니다...
4 번 째 페이지를 가져오고 있습니다...
5 번 째 페이지를 가져오고 있습니다...
6 번 째 페이지를 가져오고 있습니다...
7 번 째 페이지를 가져오고 있습니다...
8 번 째 페이지를 가져오고 있습니다...
9 번 째 페이지를 가져오고 있습니다...
10 번 째 페이지를 가져오고 있습니다...
11 번 째 페이지를 가져오고 있습니다...
12 번 째 페이지를 가져오고 있습니다...
13 번 째 페이지를 가져오고 있습니다...
14 번 째 페이지를 가져오고 있습니다...
15 번 째 페이지를 가져오고 있습니다...
16 번 째 페이지를 가져오고 있습니다...
17 번 째 페이지를 가져오고 있습니다...
18 번 째 페이지를 가져오고 있습니다...
19 번 째 페이지를 가져오고 있습니다...
20 번 째 페이지를 가져오고 있습니다...
21 번 째 페이지를 가져오고 있습니다...
22 번 째 페이지를 가져오고 있습니다...
23 번 째 페이지를 가져오고 있습니다...
24 번 째 페이지를 가져오고 있습니다...
25 번 째 페이지를 가져오고 있습니다...
26 번 째 페이지를 가져오고 있습니다...
27 번 째 페이지를 가져오고 있습니다...
28 번 째 페이지를 가져오고 있습니다...
29 번 째 페이지를 가져오고 있습니다...
30 번 째 페이지를 가져오고 있습니다...
31 번 째 페이지를 가져오고 있습니다...
32 번 째 페이지를 가져오고 있습니다...
33 번 째 페이지를 가져오고 있습니다...
34 번 째 페이지를 가져오고 있습니다...
35 번 째 페이지를 가져오고 있습니다...
36 번 째 페이지를 가져오고 있습니다...
37 번 째 페이지를 가져오고 있습니다...
38 번 째 페이지를 가져오고 있습니다...
39 번 째 페이지를 가져오고 있습니다...
40 번 째 페이지를 가져오고 있습니다...
41 번 째 페이

321 번 째 페이지를 가져오고 있습니다...
322 번 째 페이지를 가져오고 있습니다...
323 번 째 페이지를 가져오고 있습니다...
324 번 째 페이지를 가져오고 있습니다...
325 번 째 페이지를 가져오고 있습니다...
326 번 째 페이지를 가져오고 있습니다...
327 번 째 페이지를 가져오고 있습니다...
328 번 째 페이지를 가져오고 있습니다...
329 번 째 페이지를 가져오고 있습니다...
330 번 째 페이지를 가져오고 있습니다...
331 번 째 페이지를 가져오고 있습니다...
332 번 째 페이지를 가져오고 있습니다...
333 번 째 페이지를 가져오고 있습니다...
334 번 째 페이지를 가져오고 있습니다...
335 번 째 페이지를 가져오고 있습니다...
336 번 째 페이지를 가져오고 있습니다...
337 번 째 페이지를 가져오고 있습니다...
338 번 째 페이지를 가져오고 있습니다...
339 번 째 페이지를 가져오고 있습니다...
340 번 째 페이지를 가져오고 있습니다...
341 번 째 페이지를 가져오고 있습니다...
342 번 째 페이지를 가져오고 있습니다...
343 번 째 페이지를 가져오고 있습니다...
344 번 째 페이지를 가져오고 있습니다...
345 번 째 페이지를 가져오고 있습니다...
346 번 째 페이지를 가져오고 있습니다...
347 번 째 페이지를 가져오고 있습니다...
348 번 째 페이지를 가져오고 있습니다...
349 번 째 페이지를 가져오고 있습니다...
350 번 째 페이지를 가져오고 있습니다...
351 번 째 페이지를 가져오고 있습니다...
352 번 째 페이지를 가져오고 있습니다...
353 번 째 페이지를 가져오고 있습니다...
354 번 째 페이지를 가져오고 있습니다...
355 번 째 페이지를 가져오고 있습니다...
356 번 째 페이지를 가져오고 있습니다...
357 번 째 페이지를 가져오고 있습니다...
358 번 째 페이지를 가져오고 있습니다...
359 번 째 페이지를

In [34]:
aa.labeling_callrate()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


돌고있어요 2019-08-06 00:00:00
돌고있어요 2019-08-05 00:00:00


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


while문 이에요 2019-08-03 00:00:00
1
while문 이에요 2019-08-02 00:00:00
2
돌고있어요 2019-08-02 00:00:00
while문 이에요 2019-08-02 00:00:00
1
돌고있어요 2019-08-02 00:00:00
돌고있어요 2019-08-02 00:00:00
돌고있어요 2019-07-30 00:00:00
돌고있어요 2019-07-29 00:00:00
while문 이에요 2019-07-27 00:00:00
1
while문 이에요 2019-07-26 00:00:00
2
돌고있어요 2019-07-26 00:00:00


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

돌고있어요 2008-05-16 00:00:00
돌고있어요 2008-05-13 00:00:00
while문 이에요 2008-05-11 00:00:00
1
while문 이에요 2008-05-10 00:00:00
2
while문 이에요 2008-05-09 00:00:00
3
돌고있어요 2008-05-09 00:00:00
while문 이에요 2008-05-10 00:00:00
1
while문 이에요 2008-05-09 00:00:00
2
돌고있어요 2008-05-09 00:00:00
while문 이에요 2008-05-09 00:00:00
1
돌고있어요 2008-05-09 00:00:00
돌고있어요 2008-05-09 00:00:00
while문 이에요 2008-05-04 00:00:00
1
while문 이에요 2008-05-03 00:00:00
2
while문 이에요 2008-05-02 00:00:00
3
돌고있어요 2008-05-02 00:00:00
while문 이에요 2008-05-03 00:00:00
1
while문 이에요 2008-05-02 00:00:00
2
돌고있어요 2008-05-02 00:00:00
while문 이에요 2008-05-02 00:00:00
1
돌고있어요 2008-05-02 00:00:00
돌고있어요 2008-05-02 00:00:00
돌고있어요 2008-04-30 00:00:00
돌고있어요 2008-04-29 00:00:00
돌고있어요 2008-04-28 00:00:00
while문 이에요 2008-04-26 00:00:00
1
while문 이에요 2008-04-25 00:00:00
2
돌고있어요 2008-04-25 00:00:00
while문 이에요 2008-04-25 00:00:00
1
돌고있어요 2008-04-25 00:00:00
돌고있어요 2008-04-23 00:00:00
돌고있어요 2008-04-22 00:00:00
돌고있어요 2008-04-21 00:00:00
while문 이에요 2008-04-19 00:00:00
1
whil

while문 이에요 2007-09-15 00:00:00
1
while문 이에요 2007-09-14 00:00:00
2
돌고있어요 2007-09-14 00:00:00
while문 이에요 2007-09-14 00:00:00
1
돌고있어요 2007-09-14 00:00:00
돌고있어요 2007-09-12 00:00:00
돌고있어요 2007-09-11 00:00:00
돌고있어요 2007-09-10 00:00:00
while문 이에요 2007-09-08 00:00:00
1
while문 이에요 2007-09-07 00:00:00
2
돌고있어요 2007-09-07 00:00:00
while문 이에요 2007-09-07 00:00:00
1
돌고있어요 2007-09-07 00:00:00
돌고있어요 2007-09-05 00:00:00
돌고있어요 2007-09-04 00:00:00
while문 이에요 2007-09-01 00:00:00
1
while문 이에요 2007-08-31 00:00:00
2
돌고있어요 2007-08-31 00:00:00
while문 이에요 2007-08-31 00:00:00
1
돌고있어요 2007-08-31 00:00:00
돌고있어요 2007-08-28 00:00:00
돌고있어요 2007-08-27 00:00:00
돌고있어요 2007-08-21 00:00:00
돌고있어요 2007-08-20 00:00:00
while문 이에요 2007-08-18 00:00:00
1
while문 이에요 2007-08-17 00:00:00
2
돌고있어요 2007-08-17 00:00:00
while문 이에요 2007-08-17 00:00:00
1
돌고있어요 2007-08-17 00:00:00
돌고있어요 2007-08-17 00:00:00
돌고있어요 2007-08-14 00:00:00
돌고있어요 2007-08-13 00:00:00
while문 이에요 2007-08-11 00:00:00
1
while문 이에요 2007-08-10 00:00:00
2
돌고있어요 2007-08-10 0

while문 이에요 2006-12-16 00:00:00
1
while문 이에요 2006-12-15 00:00:00
2
돌고있어요 2006-12-15 00:00:00
while문 이에요 2006-12-15 00:00:00
1
돌고있어요 2006-12-15 00:00:00
돌고있어요 2006-12-15 00:00:00
돌고있어요 2006-12-12 00:00:00
돌고있어요 2006-12-11 00:00:00
while문 이에요 2006-12-09 00:00:00
1
while문 이에요 2006-12-08 00:00:00
2
돌고있어요 2006-12-08 00:00:00
while문 이에요 2006-12-08 00:00:00
1
돌고있어요 2006-12-08 00:00:00
돌고있어요 2006-12-08 00:00:00
돌고있어요 2006-12-05 00:00:00
돌고있어요 2006-12-04 00:00:00
while문 이에요 2006-12-02 00:00:00
1
while문 이에요 2006-12-01 00:00:00
2
돌고있어요 2006-12-01 00:00:00
while문 이에요 2006-12-01 00:00:00
1
돌고있어요 2006-12-01 00:00:00
돌고있어요 2006-11-29 00:00:00
돌고있어요 2006-11-28 00:00:00
돌고있어요 2006-11-27 00:00:00
while문 이에요 2006-11-25 00:00:00
1
while문 이에요 2006-11-24 00:00:00
2
돌고있어요 2006-11-24 00:00:00
돌고있어요 2006-11-22 00:00:00
돌고있어요 2006-11-21 00:00:00
돌고있어요 2006-11-20 00:00:00
while문 이에요 2006-11-18 00:00:00
1
while문 이에요 2006-11-17 00:00:00
2
돌고있어요 2006-11-17 00:00:00
while문 이에요 2006-11-17 00:00:00
1
돌고있어요 2006-11-17 0

돌고있어요 2006-04-12 00:00:00
돌고있어요 2006-04-11 00:00:00
돌고있어요 2006-04-10 00:00:00
while문 이에요 2006-04-08 00:00:00
1
while문 이에요 2006-04-07 00:00:00
2
돌고있어요 2006-04-07 00:00:00
while문 이에요 2006-04-07 00:00:00
1
돌고있어요 2006-04-07 00:00:00
돌고있어요 2006-04-04 00:00:00
돌고있어요 2006-04-03 00:00:00
while문 이에요 2006-04-01 00:00:00
1
while문 이에요 2006-03-31 00:00:00
2
돌고있어요 2006-03-31 00:00:00
돌고있어요 2006-03-28 00:00:00
돌고있어요 2006-03-27 00:00:00
while문 이에요 2006-03-25 00:00:00
1
while문 이에요 2006-03-24 00:00:00
2
돌고있어요 2006-03-24 00:00:00
while문 이에요 2006-03-24 00:00:00
1
돌고있어요 2006-03-24 00:00:00
돌고있어요 2006-03-24 00:00:00
돌고있어요 2006-03-21 00:00:00
돌고있어요 2006-03-20 00:00:00
while문 이에요 2006-03-18 00:00:00
1
while문 이에요 2006-03-17 00:00:00
2
돌고있어요 2006-03-17 00:00:00
while문 이에요 2006-03-17 00:00:00
1
돌고있어요 2006-03-17 00:00:00
돌고있어요 2006-03-17 00:00:00
돌고있어요 2006-03-14 00:00:00
돌고있어요 2006-03-13 00:00:00
while문 이에요 2006-03-11 00:00:00
1
while문 이에요 2006-03-10 00:00:00
2
돌고있어요 2006-03-10 00:00:00
while문 이에요 2006-03-10 00:

while문 이에요 2005-07-22 00:00:00
1
돌고있어요 2005-07-22 00:00:00
돌고있어요 2005-07-22 00:00:00
돌고있어요 2005-07-19 00:00:00
돌고있어요 2005-07-18 00:00:00
while문 이에요 2005-07-16 00:00:00
1
while문 이에요 2005-07-15 00:00:00
2
돌고있어요 2005-07-15 00:00:00
while문 이에요 2005-07-15 00:00:00
1
돌고있어요 2005-07-15 00:00:00
돌고있어요 2005-07-12 00:00:00
돌고있어요 2005-07-11 00:00:00
while문 이에요 2005-07-09 00:00:00
1
while문 이에요 2005-07-08 00:00:00
2
돌고있어요 2005-07-08 00:00:00
while문 이에요 2005-07-08 00:00:00
1
돌고있어요 2005-07-08 00:00:00
돌고있어요 2005-07-08 00:00:00
돌고있어요 2005-07-05 00:00:00
돌고있어요 2005-07-04 00:00:00
while문 이에요 2005-07-02 00:00:00
1
while문 이에요 2005-07-01 00:00:00
2
돌고있어요 2005-07-01 00:00:00
while문 이에요 2005-07-01 00:00:00
1
돌고있어요 2005-07-01 00:00:00
돌고있어요 2005-07-01 00:00:00
돌고있어요 2005-06-29 00:00:00
돌고있어요 2005-06-28 00:00:00
돌고있어요 2005-06-27 00:00:00
while문 이에요 2005-06-25 00:00:00
1
while문 이에요 2005-06-24 00:00:00
2
돌고있어요 2005-06-24 00:00:00
while문 이에요 2005-06-24 00:00:00
1
돌고있어요 2005-06-24 00:00:00
돌고있어요 2005-06-22 00:00:00

In [11]:
# pd.Timestamp('2005-01-01T00')

Timestamp('2005-01-01 00:00:00')

In [195]:
# df.to_csv("callrate.csv") # callrate.csv 파일로 쓰기 

In [196]:
# a = pd.read_csv("callrate.csv")

## 확인 코드

In [14]:
import pandas as pd
import numpy as np
# 잘 되었나 values_counts를 이용하여 확인 
a = pd.read_csv("callrate_fulldays.csv")
a

,Unnamed: 0,date,old_date,callrate,old_callrate,updown
0,0,2019-07-15,2019-06-15,1.73,1.74,down
1,1,2019-07-14,2019-06-15,1.73,1.74,down
2,2,2019-07-13,2019-06-15,1.73,1.74,down
3,3,2019-07-12,2019-06-12,1.74,1.75,down
4,4,2019-07-11,2019-06-11,1.75,1.75,neutrality
5,5,2019-07-10,2019-06-10,1.74,1.75,down
6,6,2019-07-09,2019-06-09,1.69,1.75,down
7,7,2019-07-08,2019-06-08,1.73,1.77,down
8,8,2019-07-07,2019-06-08,1.73,1.77,down
9,9,2019-07-06,2019-06-08,1.73,1.77,down


In [15]:
a['updown'].value_counts()

up            2178
down          2144
neutrality     987
Name: updown, dtype: int64